# Работа с метриками по Летуаль

### Импорты

In [1]:
import pandas as pd
import re
pd.options.mode.chained_assignment = None
import datetime
import numpy as np
import xlrd
import glob
from sys import exit
import sys
link_to_table = 'https://docs.google.com/spreadsheets/d/1A1TyZPeq9r2HiDi-CHfmTfRpuAQYToib1vqyTx4ZyRE/edit#gid=0'
df = pd.read_excel('/'.join(link_to_table.split('/')[:-1])+'/export?format=xlsx')
PATH, df = [dict(df.loc[x]) for x in list(df.index)], None
i = 0
while len([py for py in glob.glob(f"{PATH[i]['path_downloads']}*.xlsx")]) == 0:
    i +=1
path_downloads = PATH[i]['path_downloads']
path_algoritm = PATH[i]['path_algoritm']
sys.path.append(f'{path_algoritm}Модули')
import constants_for_marketplace_metrics_dags as const
import working_with_marketplace_metrics_dags as wwm
import maslow_working_module as mwm

#### Заказы и остатки

In [2]:
## Аналитика - Отчет по продажам - Скачать - Начать выгрузку - Ваши файлы - Скачать
path_le_new = [py for py in \
glob.glob(f"{path_downloads}sales-report-export-{mwm.dt_now.date().strftime('%Y-%m-%d')}*.xlsx")][0]

#### Цены

In [3]:
## Товары - Обновление цен, артикулов, штрихкодов - Скачать все товары - Загрузить файл
path_le_new_price_2 = 'mass-update-offers-1712232189 (2).xlsx' 

### Ретроданные

In [4]:
# Заказы

path_le_orders_metrics = path_algoritm + 'LETUAL_ORDERS/le_orders_metrics.xlsx'

le_orders_metrics = pd.read_excel(path_le_orders_metrics)
le_orders_metrics['Артикул'] = le_orders_metrics['Артикул'].astype(str)
le_orders_metrics = le_orders_metrics.iloc[:, :-1]



# Остатки

path_stocks_le_metrics = path_algoritm + 'LETUAL_ORDERS/stocks_le_metrics.xlsx'

stocks_le_metrics = pd.read_excel(path_stocks_le_metrics)
stocks_le_metrics['Артикул'] = stocks_le_metrics['Артикул'].astype(str)



# Цены

path_le_price_metrics = path_algoritm + 'LETUAL_ORDERS/le_price_metrics.xlsx'
le_price_metrics = pd.read_excel(path_le_price_metrics)
le_price_metrics['Артикул'] = le_price_metrics['Артикул'].astype(str)

In [5]:
reestr = wwm.Reestr()

In [6]:
CD = []
for row in le_price_metrics.columns[-5:]:
    if le_price_metrics[row].isna().sum() == 0:
        CD.append(row)

for row in CD:
    le_price_metrics = le_price_metrics.drop(row, axis=1)
    le_orders_metrics = le_orders_metrics.drop(row, axis=1)

In [7]:
CD = []
for row in le_orders_metrics.columns[-5:]:
    if le_orders_metrics[row].isna().sum() == 0:
        CD.append(row)

for row in CD:
    le_orders_metrics = le_orders_metrics.drop(row, axis=1)

In [8]:
# файл с себестоимостью (общий)

path_cost_price = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/Себестоимость /cost_price 2.0.xlsx'

cp = pd.read_excel(path_cost_price)
cp['Артикул'] = cp['Артикул'].astype(str)

In [9]:
# файл с признаком "РЦ" (общий)

path_rc = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/РЦ/РЦ_обработанные.xlsx'

rc = pd.read_excel(path_rc)
rc['Артикул'] = rc['Артикул'].astype(str)

In [10]:
# файл с признаком "В резерве" (общий)

path_vr = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/В резерве/В резерве_обработанные.xlsx'

vr = pd.read_excel(path_vr)
vr['Артикул'] = vr['Артикул'].astype(str)

In [11]:


# файл со Статусами

path_st = wwm.path_st
st = pd.read_excel(path_st)
st['Артикул'] = st['Артикул'].astype(str)

In [12]:
st

,Артикул,Статус
0,16920,текущий
1,16930,текущий
2,16950,сезонка
3,18704,отказ
4,18705,отказ
...,...,...
1303,50976,текущий Дарья
1304,g0901,текущий Дарья
1305,50458,товары в пути
1306,50459,товары в пути


In [13]:
def dt_col(data):
    date_col = []
    for row in data.columns:
        try:
            pd.to_datetime(row)
            date_col.append(row)
        except:
            None
    return date_col

In [14]:
pd.read_excel(path_le_new)

,Дата,Артикул Алькора,Артикул,Бренд,Наименование товара\nдля отображения на сайте,Категория товара,Штрихкод товара,"Транзит, шт.","Товаров продано, шт.","Продано на сумму, руб.","Текущий остаток, шт."
0,2024-04-08,MPL165541,22400,REVOLUT,Апельсиновые палочки для маникюра,Палочки для маникюра,4638290217042,3,0,0,3
1,2024-04-08,MPL166486,19600,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216410,4,0,0,4
2,2024-04-08,MPL166487,19601,SHARK FIT,Набор ленточных эспандеров для фитнеса,Эспандер-лента,4638290216441,14,0,0,58
3,2024-04-08,MPL166488,19602,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216472,2,0,0,41
4,2024-04-08,MPL166489,19603,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216502,2,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...
2433,2024-03-25,MPL291125,H0107,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4697259245847,0,0,0,10
2434,2024-03-25,MPL291174,H0140,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4646873394691,0,0,0,10
2435,2024-03-25,MPL291175,H0141,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4687472879233,0,0,0,17
2436,2024-03-25,MPL291177,H0142,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4628632487677,0,0,0,9


In [15]:
le = pd.read_excel(path_le_new)
le['Артикул'] = le['Артикул'].astype(str)
le['Дата'] = list(map(lambda x: pd.to_datetime(x).strftime('%d-%B-%Y'), list(le['Дата'].values)))

In [16]:
le

,Дата,Артикул Алькора,Артикул,Бренд,Наименование товара\nдля отображения на сайте,Категория товара,Штрихкод товара,"Транзит, шт.","Товаров продано, шт.","Продано на сумму, руб.","Текущий остаток, шт."
0,08-April-2024,MPL165541,22400,REVOLUT,Апельсиновые палочки для маникюра,Палочки для маникюра,4638290217042,3,0,0,3
1,08-April-2024,MPL166486,19600,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216410,4,0,0,4
2,08-April-2024,MPL166487,19601,SHARK FIT,Набор ленточных эспандеров для фитнеса,Эспандер-лента,4638290216441,14,0,0,58
3,08-April-2024,MPL166488,19602,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216472,2,0,0,41
4,08-April-2024,MPL166489,19603,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216502,2,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...
2433,25-March-2024,MPL291125,H0107,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4697259245847,0,0,0,10
2434,25-March-2024,MPL291174,H0140,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4646873394691,0,0,0,10
2435,25-March-2024,MPL291175,H0141,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4687472879233,0,0,0,17
2436,25-March-2024,MPL291177,H0142,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4628632487677,0,0,0,9


In [17]:
le_price_2 = pd.read_excel(path_downloads + path_le_new_price_2)
le_price_2 = le_price_2[[le_price_2.columns[0]]+[le_price_2.columns[5]] +[le_price_2.columns[-5]]]
le_price_2.columns = list(le_price_2.loc[2].values)
le_price_2 = le_price_2.iloc[4:, :].reset_index(drop=True)
le_price_2 = le_price_2.dropna()
le_price_2

,Артикул,Дата создания,"Цена после скидки, руб"
0,50285,23/03/2023,640
1,B0156,26/05/2023,303
2,B0157,26/05/2023,296
3,26001,07/06/2023,250
4,26005,07/06/2023,250
...,...,...,...
213,26004,07/06/2023,250
214,J0561,09/06/2023,500
215,28609,11/07/2023,370
218,J0004,23/03/2023,180


In [18]:
le

,Дата,Артикул Алькора,Артикул,Бренд,Наименование товара\nдля отображения на сайте,Категория товара,Штрихкод товара,"Транзит, шт.","Товаров продано, шт.","Продано на сумму, руб.","Текущий остаток, шт."
0,08-April-2024,MPL165541,22400,REVOLUT,Апельсиновые палочки для маникюра,Палочки для маникюра,4638290217042,3,0,0,3
1,08-April-2024,MPL166486,19600,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216410,4,0,0,4
2,08-April-2024,MPL166487,19601,SHARK FIT,Набор ленточных эспандеров для фитнеса,Эспандер-лента,4638290216441,14,0,0,58
3,08-April-2024,MPL166488,19602,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216472,2,0,0,41
4,08-April-2024,MPL166489,19603,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216502,2,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...
2433,25-March-2024,MPL291125,H0107,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4697259245847,0,0,0,10
2434,25-March-2024,MPL291174,H0140,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4646873394691,0,0,0,10
2435,25-March-2024,MPL291175,H0141,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4687472879233,0,0,0,17
2436,25-March-2024,MPL291177,H0142,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4628632487677,0,0,0,9


In [19]:
le[['Артикул', 'Бренд', 'Наименование товара\nдля отображения на сайте', 'Категория товара', 'Штрихкод товара']]

,Артикул,Бренд,Наименование товара\nдля отображения на сайте,Категория товара,Штрихкод товара
0,22400,REVOLUT,Апельсиновые палочки для маникюра,Палочки для маникюра,4638290217042
1,19600,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216410
2,19601,SHARK FIT,Набор ленточных эспандеров для фитнеса,Эспандер-лента,4638290216441
3,19602,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216472
4,19603,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216502
...,...,...,...,...,...
2433,H0107,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4697259245847
2434,H0140,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4646873394691
2435,H0141,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4687472879233
2436,H0142,HAMSA YOGA,"Блок для йоги, Кирпичик для растяжки",Блок для йоги,4628632487677


In [20]:
le_name = le[['Артикул', 'Бренд', 'Наименование товара\nдля отображения на сайте', 'Категория товара', 'Штрихкод товара']].copy()
le_name['Артикул'] = le_name['Артикул'].astype(str)
le_name = le_name.drop_duplicates().reset_index(drop=True)
if len(le_name) - len(list(set(le_name['Артикул']))) != 0:
    le_name = 'error'
    print(le_name)
    
le_name = le_name.query(f"Артикул not in {list(set(le_orders_metrics['Артикул']))}").reset_index(drop=True)
le_name = le_name.rename(columns={'Наименование товара\nдля отображения на сайте': 
                                  'Наименование товара\nдля отображения на сайт'})
if len(le_name) > 0:
    le_orders_metrics = pd.concat([le_orders_metrics, le_name], ignore_index=True)
    le_orders_metrics = le_orders_metrics.fillna(0)

In [21]:
le_orders = le[['Дата', 'Артикул', 'Товаров продано, шт.']].copy()
le_stocks = le[['Дата', 'Артикул', 'Текущий остаток, шт.']].copy()






le_price = le[['Дата', 'Артикул', 'Товаров продано, шт.', 'Продано на сумму, руб.']].copy()
le_price['Цена'] = le_price['Продано на сумму, руб.'] / le_price['Товаров продано, шт.']
le_price = le_price[['Дата', 'Артикул', 'Цена']]
le_price = le_price.drop_duplicates().reset_index(drop=True)
le_price = le_price[le_price['Цена'] > 0].reset_index(drop=True)


In [22]:
def rep(y):
    try:
        if float(y) <= 0 :
            y = None
    except:
        y = None
    return y

In [23]:
R = list(le_orders['Дата'].unique())[::-1]

for date in R:
    if R.index(date) == 0:
        le_orders_DF = le_orders[le_orders['Дата']==date].iloc[:, 1:].rename(columns={le_orders.columns[-1]: date}).groupby('Артикул')[date].sum().reset_index(drop=False).copy()
        
        le_stocks_DF = le_stocks[le_stocks['Дата']==date].iloc[:, 1:].rename(columns={le_stocks.columns[-1]: date}).groupby('Артикул')[date].sum().reset_index(drop=False).copy()
        
        le_price_DF = le_price[le_price['Дата']==date].iloc[:, 1:].rename(columns={le_price.columns[-1]: date}).groupby('Артикул')[date].sum().reset_index(drop=False).copy()
    else:
        le_orders_df = le_orders[le_orders['Дата']==date].iloc[:, 1:].rename(columns={le_orders.columns[-1]: date}).groupby('Артикул')[date].sum().reset_index(drop=False).copy()
        le_orders_DF = le_orders_DF.merge(le_orders_df, on='Артикул', how='outer')
        
        le_stocks_df = le_stocks[le_stocks['Дата']==date].iloc[:, 1:].rename(columns={le_stocks.columns[-1]: date}).groupby('Артикул')[date].sum().reset_index(drop=False).copy()
        le_stocks_DF = le_stocks_DF.merge(le_stocks_df, on='Артикул', how='outer')
        
        le_price_df = le_price[le_price['Дата']==date].iloc[:, 1:].rename(columns={le_price.columns[-1]: date}).groupby('Артикул')[date].sum().reset_index(drop=False).copy()
        le_price_DF = le_price_DF.merge(le_price_df, on='Артикул', how='outer')
        
        
        
le_orders_df = None
le_stocks_df = None
le_price_df = None
le_orders_DF = le_orders_DF.fillna(0)
le_stocks_DF = le_stocks_DF.fillna(0)
le_orders = None
le_stocks = None
le_price = None
le_orders = le_orders_DF
le_stocks = le_stocks_DF
le_price = le_price_DF


In [24]:
le_price

,Артикул,25-March-2024,26-March-2024,27-March-2024,28-March-2024,29-March-2024,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,19201,175.0,175.0,175.0,NaN,NaN,NaN,NaN,175.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19600,310.0,310.0,310.0,310.0,310.0,310.0,NaN,310.000000,310.0,310.0,NaN,310.0,NaN,NaN,NaN
2,19601,333.0,333.0,333.0,333.0,333.0,333.0,NaN,333.000000,333.0,333.0,NaN,NaN,333.0,333.0,NaN
3,19602,390.0,390.0,390.0,390.0,390.0,390.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19603,389.0,389.0,NaN,389.0,389.0,389.0,389.0,382.333333,NaN,389.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,28608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,665.0,NaN,NaN,NaN
104,H0077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2290.0,NaN,NaN,NaN
105,19807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172.0,NaN
106,H0075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2290.0,NaN


In [25]:
if 'index' in le_orders_metrics.columns:
    le_orders_metrics = le_orders_metrics.drop('index', axis=1)
    
le_orders_metrics = wwm.data_columns_sorted(le_orders_metrics)

In [26]:
for date in R:
    
    if date not in list(set(le_orders_metrics.columns)):
        le_orders_metrics = le_orders_metrics.merge(le_orders[['Артикул', date]], on='Артикул', how='outer')
    else:
        None
        
    if date not in list(set(stocks_le_metrics.columns)):
        stocks_le_metrics = stocks_le_metrics.merge(le_stocks[['Артикул', date]], on='Артикул', how='outer')
    else:
        None
        
    if date not in list(set(le_price_metrics.columns)):
        le_price_metrics = le_price_metrics.merge(le_price[['Артикул', date]], on='Артикул', how='outer')
    else:
        None

In [27]:
# le_orders_metrics['23-January-2024'] = 0

In [28]:
le_orders_metrics

,Артикул,Бренд,Наименование товара\nдля отображения на сайт,Категория товара,Штрихкод товара,12-January-2023,13-January-2023,14-January-2023,15-January-2023,16-January-2023,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,22400,REVOLUT,"Апельсиновые палочки для маникюра, 100шт",палочки для маникюра,4638290217042,0,0,0,0,0,...,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19600,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216410,0,0,0,0,0,...,7,0,1,3.0,2.0,0.0,1.0,0.0,0.0,0.0
2,19601,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216441,0,0,0,0,0,...,4,0,5,1.0,1.0,0.0,0.0,3.0,1.0,0.0
3,19603,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216502,0,0,0,0,0,...,10,1,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,28600,SHARK FIT,"Валик большой массажный, синий",Валик для фитнеса,4638290217134,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,G0505,SSY,Набор игрушек для собак мелких пород,Игрушка,4647533992240,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183,27741,SHARK FIT,Обруч разборный,Обруч,4626948863161,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184,27751,SHARK FIT,"Утяжелители спортивные черные, 2x2,0 кг.",Утяжелитель,4631688950781,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,G0616,BIRDHOUSE,Коврик придверный противоскользящий 60x90 синий,Придверный коврик,4659295292278,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
le_df = le_orders_metrics.copy()
le_df = le_df.merge(stocks_le_metrics[['Артикул', stocks_le_metrics.columns[-1]]].rename(columns={stocks_le_metrics.columns[-1]: 'Остаток'}).copy(), on='Артикул', how='left')
le_df = le_df.merge(le_price_metrics[['Артикул', le_price_metrics.columns[-1]]].rename(columns={le_price_metrics.columns[-1]: 'Цена'}).copy(), on='Артикул', how='left')

In [30]:
# le_df = le_df[list(le_df.columns[:le_df.columns.get_loc(dt_col(le_df)[0])]) + 
#           list(map(lambda x: x.strftime('%d-%B-%Y'), sorted(list(map(lambda x: pd.to_datetime(x), dt_col(sbm)))))) + 
#           list(sbm.columns[sbm.columns.get_loc(dt_col(sbm)[-1]) + 1:])]

In [31]:
# l = ['Итого'] + [None] * len(le_df.columns[1:le_df.columns.get_loc(dt_col(le_df)[0])]) + list(map(lambda x: le_df[x].sum(), dt_col(le_df))) 
# le_df.loc[len(le_df)] = l + (len(le_df.columns) - len(l)) * [None]

In [32]:
le_df

,Артикул,Бренд,Наименование товара\nдля отображения на сайт,Категория товара,Штрихкод товара,12-January-2023,13-January-2023,14-January-2023,15-January-2023,16-January-2023,...,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Цена
0,22400,REVOLUT,"Апельсиновые палочки для маникюра, 100шт",палочки для маникюра,4638290217042,0,0,0,0,0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
1,19600,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216410,0,0,0,0,0,...,1,3.0,2.0,0.0,1.0,0.0,0.0,0.0,4.0,NaN
2,19601,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216441,0,0,0,0,0,...,5,1.0,1.0,0.0,0.0,3.0,1.0,0.0,58.0,NaN
3,19603,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4638290216502,0,0,0,0,0,...,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
4,28600,SHARK FIT,"Валик большой массажный, синий",Валик для фитнеса,4638290217134,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,G0505,SSY,Набор игрушек для собак мелких пород,Игрушка,4647533992240,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,NaN
183,27741,SHARK FIT,Обруч разборный,Обруч,4626948863161,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,NaN
184,27751,SHARK FIT,"Утяжелители спортивные черные, 2x2,0 кг.",Утяжелитель,4631688950781,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,NaN
185,G0616,BIRDHOUSE,Коврик придверный противоскользящий 60x90 синий,Придверный коврик,4659295292278,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,NaN


In [33]:
le_df.loc[len(le_df)-1] = ['Итого', None, None, None, None] + list(map(lambda x: le_df[x].sum(), list(le_df.columns[5:])))

In [34]:
le_df['Цена'].unique()

array([nan,  0.])

In [35]:
le_df.loc[0]['Цена'] 

nan

In [36]:
len(le_df['Цена'].unique())

2

In [37]:
for i in range(len(le_df)):
    art = le_df.loc[i]['Артикул']
    if str(le_df.loc[i]['Цена']) == 'nan':
        if art in list(set(le_price_2['Артикул'])):
            le_df['Цена'].loc[i] = le_price_2[le_price_2['Артикул']==art][le_price_2.columns[-1]].values[0]

In [38]:
len(le_df['Цена'].unique())

73

In [39]:
le_df

,Артикул,Бренд,Наименование товара\nдля отображения на сайт,Категория товара,Штрихкод товара,12-January-2023,13-January-2023,14-January-2023,15-January-2023,16-January-2023,...,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Цена
0,22400,REVOLUT,"Апельсиновые палочки для маникюра, 100шт",палочки для маникюра,4.638290e+12,0,0,0,0,0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,79
1,19600,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,0,0,0,0,0,...,1,3.0,2.0,0.0,1.0,0.0,0.0,0.0,4.0,310
2,19601,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,0,0,0,0,0,...,5,1.0,1.0,0.0,0.0,3.0,1.0,0.0,58.0,333
3,19603,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,0,0,0,0,0,...,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,389
4,28600,SHARK FIT,"Валик большой массажный, синий",Валик для фитнеса,4.638290e+12,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,G0505,SSY,Набор игрушек для собак мелких пород,Игрушка,4.647534e+12,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,NaN
183,27741,SHARK FIT,Обруч разборный,Обруч,4.626949e+12,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,NaN
184,27751,SHARK FIT,"Утяжелители спортивные черные, 2x2,0 кг.",Утяжелитель,4.631689e+12,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,NaN
185,G0616,BIRDHOUSE,Коврик придверный противоскользящий 60x90 синий,Придверный коврик,4.659295e+12,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,NaN


In [40]:
Q = []

for i in range(len(le_price_metrics)):
    E = []
    for row in list(le_price_metrics.columns[1:])[::-1]:
        if str(le_price_metrics.loc[i][row]) != 'nan':
            E.append(le_price_metrics.loc[i][row])
    if len(E) > 0:
        Q.append(E[0])
    else:
        Q.append(None)

In [41]:
le_price_dop = le_price_metrics[['Артикул']].copy()
le_price_dop['Цена'] = Q

In [42]:
le_price_dop = le_price_dop.dropna().reset_index(drop=True)

In [43]:
le_price_dop['Артикул'] = le_price_dop['Артикул'].astype(str)

In [44]:
for i in range(len(le_df)):
    art = le_df.loc[i]['Артикул']
    if str(le_df.loc[i]['Цена']) == 'nan':
        if art in list(set(le_price_dop['Артикул'])):
            le_df['Цена'].loc[i] = le_price_dop[le_price_dop['Артикул']==art][le_price_dop.columns[-1]].values[0]

In [45]:
for i in range(len(le_df)):
    
    art = le_df.loc[i]['Артикул']
    
    if art in list(set(le_price_2['Артикул'])):
        if le_df.loc[i:i]['Цена'].isna()[i] == True:
            le_df['Цена'].loc[i] = le_price_2[le_price_2['Артикул']==art][le_price_2.columns[-1]].values[0]

In [46]:
le_df = le_df.merge(cp, on='Артикул', how='left')

In [47]:
le_df['Цена'] = le_df['Цена'].astype(float)

In [48]:
def le_nac(data):
    Q = []
    for i in range(len(data)):
        if data.loc[i]['Бренд'] in ['REVOLUT', 'SHARK FIT']:
            Q.append(data.loc[i]['Цена'] / 100 * 20)
        elif data.loc[i]['Бренд'] == None:
            Q.append(None)
        else:
            Q.append(data.loc[i]['Цена'] / 100 * 40)
    data['Стоимость услуг, руб (40%)'] = Q
    data['Цена_НДС'] = data['Цена'] * 0.2 / 1.2
    data['Себестоимость_НДС'] = data['Себестоимость'] * 0.2 / 1.2
    data['Стоимость услуг, руб (40%)_НДС'] = data['Стоимость услуг, руб (40%)'] * 0.2 / 1.2
    data['НДС'] = data['Цена_НДС'] - data['Себестоимость_НДС'] - data['Стоимость услуг, руб (40%)_НДС']
    data['Цена_ННП'] = data['Цена'] - data['Цена_НДС']
    data['Себестоимость_ННП'] = data['Себестоимость'] - data['Себестоимость_НДС']
    data['Стоимость услуг, руб (40%)_НДС'] = data['Стоимость услуг, руб (40%)_НДС'] - data['Стоимость услуг, руб (40%)_НДС']
    data['ННП'] = (data['Цена_ННП'] - data['Себестоимость_ННП'] - data['Стоимость услуг, руб (40%)_НДС']) * 0.2

    data['Налог'] = data['НДС'] + data['ННП']

    data['% налога'] = data['Налог'] * 100 / data['Цена']
    data['Прибыль'] = data['Цена'] - data['Стоимость услуг, руб (40%)'] - data['Себестоимость']- data['Налог']
    data['Наценка'] = data['Прибыль'] / data['Себестоимость'] * 100

    return data


In [49]:
le_df = le_nac(le_df)

In [50]:
le_df = le_df.merge(st, on='Артикул', how='left')

In [51]:
le_df = le_df.merge(rc, on='Артикул', how='left')
le_df = le_df.merge(vr, on='Артикул', how='left')

In [52]:
le_df22 = le_df.copy()

In [53]:
le_df = le_df[list(le_df.columns[:5])+ ['Статус'] + dt_col(le_df) + ['Остаток', 'Цена', 'Наценка', 'Остаток на РЦ', 'В резерве']]

In [54]:
le_df['Остаток на РЦ'].loc[len(le_df)-1] = le_df['Остаток на РЦ'].sum()
le_df['В резерве'].loc[len(le_df)-1] = le_df['В резерве'].sum()

In [55]:
le_df

,Артикул,Бренд,Наименование товара\nдля отображения на сайт,Категория товара,Штрихкод товара,Статус,12-January-2023,13-January-2023,14-January-2023,15-January-2023,...,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Цена,Наценка,Остаток на РЦ,В резерве
0,22400,REVOLUT,"Апельсиновые палочки для маникюра, 100шт",палочки для маникюра,4.638290e+12,отказ,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,3.0,79.0,1.530847,NaN,NaN
1,19600,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,текущий,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,4.0,310.0,65.057647,4.0,NaN
2,19601,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,текущий,0,0,0,0,...,0.0,0.0,3.0,1.0,0.0,58.0,333.0,35.424203,636.0,4.0
3,19603,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,текущий,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,3.0,389.0,53.372832,8.0,2.0
4,28600,SHARK FIT,"Валик большой массажный, синий",Валик для фитнеса,4.638290e+12,текущий,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,23.0,705.0,30.698265,1509.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,G0505,SSY,Набор игрушек для собак мелких пород,Игрушка,4.647534e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,20.0,NaN,NaN,174.0,1.0
183,27741,SHARK FIT,Обруч разборный,Обруч,4.626949e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,30.0,NaN,NaN,101.0,8.0
184,27751,SHARK FIT,"Утяжелители спортивные черные, 2x2,0 кг.",Утяжелитель,4.631689e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,20.0,NaN,NaN,259.0,42.0
185,G0616,BIRDHOUSE,Коврик придверный противоскользящий 60x90 синий,Придверный коврик,4.659295e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,10.0,NaN,NaN,30.0,NaN


In [56]:
le_df = le_df[list(le_df.columns[:le_df.columns.get_loc(dt_col(le_df)[0])]) + 
          list(map(lambda x: x.strftime('%d-%B-%Y'), sorted(list(map(lambda x: pd.to_datetime(x), dt_col(le_df))))))[-31:] + 
          list(le_df.columns[le_df.columns.get_loc(dt_col(le_df)[-1]) + 1:])]

In [57]:
le_df

,Артикул,Бренд,Наименование товара\nдля отображения на сайт,Категория товара,Штрихкод товара,Статус,09-March-2024,10-March-2024,11-March-2024,12-March-2024,...,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Цена,Наценка,Остаток на РЦ,В резерве
0,22400,REVOLUT,"Апельсиновые палочки для маникюра, 100шт",палочки для маникюра,4.638290e+12,отказ,0,0,1,2,...,0.0,0.0,0.0,0.0,0.0,3.0,79.0,1.530847,NaN,NaN
1,19600,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,текущий,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,4.0,310.0,65.057647,4.0,NaN
2,19601,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,текущий,1,0,3,4,...,0.0,0.0,3.0,1.0,0.0,58.0,333.0,35.424203,636.0,4.0
3,19603,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,текущий,0,0,0,3,...,0.0,0.0,0.0,0.0,0.0,3.0,389.0,53.372832,8.0,2.0
4,28600,SHARK FIT,"Валик большой массажный, синий",Валик для фитнеса,4.638290e+12,текущий,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,23.0,705.0,30.698265,1509.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,G0505,SSY,Набор игрушек для собак мелких пород,Игрушка,4.647534e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,20.0,NaN,NaN,174.0,1.0
183,27741,SHARK FIT,Обруч разборный,Обруч,4.626949e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,30.0,NaN,NaN,101.0,8.0
184,27751,SHARK FIT,"Утяжелители спортивные черные, 2x2,0 кг.",Утяжелитель,4.631689e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,20.0,NaN,NaN,259.0,42.0
185,G0616,BIRDHOUSE,Коврик придверный противоскользящий 60x90 синий,Придверный коврик,4.659295e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,10.0,NaN,NaN,30.0,NaN


In [58]:
def weekly_amount1(data):
    
    data1 = data.copy()

    z = dt_col(data)

    for i in range(len(z)):
        if pd.to_datetime(z[i]).dayofweek == 6 and i > 5:
            data1.insert((data1.columns.get_loc(z[i]))+1, f"{pd.to_datetime(z[i-6]):%d-%B}-{pd.to_datetime(z[i]):%d-%B}", 0)



    data = data1

    R = []
    for i in range(len(list(data.columns))):
        try:
            if len((re.findall('(\d+)',list(data.columns)[i]))[1]) != 4:
                R.append(list(data.columns)[i])

        except:
            None

    for row in R:
        T = []
        for i in range(len(data)):
            T.append((data.iloc[i, data.columns.get_loc(row)-7:data.columns.get_loc(row)]).sum())

        data[row] = T
        
    
    data['Артикул'] = data['Артикул'].astype(str)
        
    return data

In [59]:
le_df = weekly_amount1(le_df)

In [60]:
le_df

,Артикул,Бренд,Наименование товара\nдля отображения на сайт,Категория товара,Штрихкод товара,Статус,09-March-2024,10-March-2024,11-March-2024,12-March-2024,...,05-April-2024,06-April-2024,07-April-2024,01-April-07-April,08-April-2024,Остаток,Цена,Наценка,Остаток на РЦ,В резерве
0,22400,REVOLUT,"Апельсиновые палочки для маникюра, 100шт",палочки для маникюра,4.638290e+12,отказ,0,0,1,2,...,0.0,0.0,0.0,1.0,0.0,3.0,79.0,1.530847,NaN,NaN
1,19600,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,текущий,0,0,0,0,...,1.0,0.0,0.0,7.0,0.0,4.0,310.0,65.057647,4.0,NaN
2,19601,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,текущий,1,0,3,4,...,0.0,3.0,1.0,11.0,0.0,58.0,333.0,35.424203,636.0,4.0
3,19603,SHARK FIT,Эспандер ленточный - набор резинок для фитнеса...,Эспандер-лента,4.638290e+12,текущий,0,0,0,3,...,0.0,0.0,0.0,4.0,0.0,3.0,389.0,53.372832,8.0,2.0
4,28600,SHARK FIT,"Валик большой массажный, синий",Валик для фитнеса,4.638290e+12,текущий,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,23.0,705.0,30.698265,1509.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,G0505,SSY,Набор игрушек для собак мелких пород,Игрушка,4.647534e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,20.0,NaN,NaN,174.0,1.0
183,27741,SHARK FIT,Обруч разборный,Обруч,4.626949e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,30.0,NaN,NaN,101.0,8.0
184,27751,SHARK FIT,"Утяжелители спортивные черные, 2x2,0 кг.",Утяжелитель,4.631689e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,20.0,NaN,NaN,259.0,42.0
185,G0616,BIRDHOUSE,Коврик придверный противоскользящий 60x90 синий,Придверный коврик,4.659295e+12,текущий Дарья,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,10.0,NaN,NaN,30.0,NaN


In [61]:
rc_irc = list(le_df[['Артикул']]\
         .merge(pd.read_excel(path_downloads + 'РЦ Иркутск.xlsx', 
                dtype={'Артикул': 'str'}).groupby('Артикул')['Остаток РЦ Иркутск'].sum()\
                .reset_index(drop=False), 
                on='Артикул', 
                how='left')['Остаток РЦ Иркутск'])

le_df.insert(le_df.columns.get_loc('Остаток на РЦ')+1, 'Остаток РЦ Иркутск', rc_irc)

In [62]:
len(le_orders_metrics) - len(list(set(le_orders_metrics['Артикул'])))

0

In [63]:
len(le_price_metrics) - len(list(set(le_price_metrics['Артикул'])))

0

In [64]:
len(stocks_le_metrics) - len(list(set(stocks_le_metrics['Артикул'])))

0

In [65]:
len(le_df) - len(list(set(le_df['Артикул'])))

0

In [66]:
le_orders_metrics = wwm.rename_data_from_reestr(le_orders_metrics, 
                                                  'Наименование товара\nдля отображения на сайт', 
                                                  reestr=reestr)
le_df = wwm.rename_data_from_reestr(le_df, 
                                    'Наименование товара\nдля отображения на сайт', 
                                    reestr=reestr)

In [67]:
stocks_le_metrics.to_excel(path_stocks_le_metrics, index=False)

In [68]:
le_price_metrics.to_excel(path_le_price_metrics, index=False)

In [69]:
le_orders_metrics.to_excel(path_le_orders_metrics, index=False)

## Расчет и добавление "выручки", "доли", "итоговой наценки"

In [70]:
dfn = None

dfn = le_df22[['Артикул', 'Прибыль', 'Себестоимость', 'Наценка', dt_col(le_df22)[-2]]]\
     .rename(columns={dt_col(le_df22)[-2]: 'Продажи'})

dfn = dfn[(dfn['Себестоимость'].astype(str).isin(['0', 'nan', '0.0'])==False)\
        &(dfn['Продажи']>0)]\
     .reset_index(drop=True)

ITOG_NAC = round((((dfn['Прибыль'] * dfn['Продажи']).sum() / (dfn['Себестоимость'] *  dfn['Продажи']).sum()) * 100), 2)

# dfn = None

if str(ITOG_NAC) == 'nan':
    ITOG_NAC = 0

In [71]:
ITOG_NAC

37.54

In [72]:
le_df['Выручка'] = le_df['Цена'] * le_df[dt_col(le_df)[-2]] 
le_df['Выручка'].loc[len(le_df)-1] = 0
le_df['Выручка'] = le_df['Выручка'].astype(float)
s = le_df['Выручка'].sum()
le_df['Доля выручки'] = (le_df['Выручка'] * 100) / s
le_df['Выручка'].loc[len(le_df)-1] = s
le_df['Итого наценка'] = None
# e = WB_full_df['Итого наценка'].sum()
# e = le_df[le_df['Итого наценка']!=np.inf]['Итого наценка'].sum()
le_df['Итого наценка'].loc[len(le_df)-1] = ITOG_NAC

## прибыль с единицы товара

In [73]:
le_df = le_df.merge(le_df22[['Артикул', 
                        'Прибыль']]\
              .rename(columns={'Прибыль': 'прибыль с единицы товара'}),
              on='Артикул', 
              how='left') 
le_df['прибыль с единицы товара'] = le_df['прибыль с единицы товара'].round(2)

## Сохранение наценки

In [74]:
le_markup_metrics = pd.read_excel(wwm.path_le_markup_metrics,
                                  dtype={'Артикул': 'str'})
if len(le_markup_metrics) - len(list(set(le_markup_metrics['Артикул']))) != 0:
    print('Неуникальные артикулы в le_markup_metrics')
    exit()
le_markup_new = le_df.iloc[:-1, :][['Артикул', 'Наценка']]\
                .rename(columns={'Наценка': const.dt_now.strftime('%d-%B-%Y')}).copy()
if len(le_markup_new) - len(list(set(le_markup_new['Артикул']))) != 0:
    print('Неуникальные артикулы в le_markup_new')
    exit()
col = wwm.dt_col(le_markup_new)[0]
if col in le_markup_metrics.columns:
    le_markup_metrics = le_markup_metrics.drop(col, axis=1)
    le_markup_metrics = le_markup_metrics.merge(le_markup_new, on='Артикул', how='outer')
else:
    le_markup_metrics = le_markup_metrics.merge(le_markup_new, on='Артикул', how='outer')
if len(le_markup_metrics) - len(list(set(le_markup_metrics['Артикул']))) != 0:
    print('Неуникальные артикулы в le_markup_metrics')
    exit()
le_markup_metrics.to_excel(wwm.path_le_markup_metrics, index=False)

In [75]:
def unloading():
    
    with pd.ExcelWriter(path_algoritm + "Все файлы для ежедневной презентации/full_LE_df.xlsx") as writer:


        le_orders_metrics.to_excel(writer, sheet_name="le_orders_metrics", index=False)
        for column in le_orders_metrics:
            column_width = max(le_orders_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = le_orders_metrics.columns.get_loc(column)
            writer.sheets['le_orders_metrics'].set_column(col_idx, col_idx, column_width)



        stocks_le_metrics.to_excel(writer, sheet_name="stocks_le_metrics", index=False)
        for column in stocks_le_metrics:
            column_width = max(stocks_le_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = stocks_le_metrics.columns.get_loc(column)
            writer.sheets['stocks_le_metrics'].set_column(col_idx, col_idx, column_width)



        le_price_metrics.to_excel(writer, sheet_name="le_price_metrics", index=False)
        for column in le_price_metrics:
            column_width = max(le_price_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = le_price_metrics.columns.get_loc(column)
            writer.sheets['le_price_metrics'].set_column(col_idx, col_idx, column_width)






        le_df.to_excel(writer, sheet_name="le_full_df", index=False)
        for column in le_df:
            column_width = max(le_df[column].astype(str).map(len).max() + 3, len(column))
            col_idx = le_df.columns.get_loc(column)
            writer.sheets['le_full_df'].set_column(col_idx, col_idx, column_width)



    writer.save()

In [76]:
try:
    unloading()
except:
    unloading()

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [77]:
list(le_df.columns)

['Артикул',
 'Бренд',
 'Наименование товара\nдля отображения на сайт',
 'Категория товара',
 'Штрихкод товара',
 'Статус',
 '09-March-2024',
 '10-March-2024',
 '11-March-2024',
 '12-March-2024',
 '13-March-2024',
 '14-March-2024',
 '15-March-2024',
 '16-March-2024',
 '17-March-2024',
 '11-March-17-March',
 '18-March-2024',
 '19-March-2024',
 '20-March-2024',
 '21-March-2024',
 '22-March-2024',
 '23-March-2024',
 '24-March-2024',
 '18-March-24-March',
 '25-March-2024',
 '26-March-2024',
 '27-March-2024',
 '28-March-2024',
 '29-March-2024',
 '30-March-2024',
 '31-March-2024',
 '25-March-31-March',
 '01-April-2024',
 '02-April-2024',
 '03-April-2024',
 '04-April-2024',
 '05-April-2024',
 '06-April-2024',
 '07-April-2024',
 '01-April-07-April',
 '08-April-2024',
 'Остаток',
 'Цена',
 'Наценка',
 'Остаток на РЦ',
 'Остаток РЦ Иркутск',
 'В резерве',
 'Выручка',
 'Доля выручки',
 'Итого наценка',
 'прибыль с единицы товара']

In [78]:
exit()

SystemExit: 

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
